# Текущий план
- попробовать оформить и обучить отдельно текстовый регрессор (классификатор) на spacy - можно сделать в отдельном файле test.ipynb, там уже описано как делать, и мы имеем достаточно примеров
- добавляем вектора из spacy

Идея: см. какие теги нашлись, проверяем, какое количество просмотров предсказала бы модель для худшего и лучшего сценария, и находим, где между минимумом и максимумом находится наша точка.

# Описание задачи

**Цель**: сделать веб-приложение, помогающее авторам и редакторам IT-публикаций подбирать к ним удачные заголовки.

**Задача-минимум**: создать сервис, который учит пользователя (автора, редактора) использовать разумные подходы для составления говорящего заголовка. По такому заголовку читатель понимает, какую пользу он получит от прочтения статьи.

**Идея MVP**. Проблему можно сформулировать в виде задачи распознавания именованных сущностей (англ. [Named Entity Recognition](https://en.wikipedia.org/wiki/Named-entity_recognition), NER). Распознанные именованные сущности можно далее вмесе с токенизированным текстом использовать для выставления условного балла от 0 до 10, позволяющего автору быстро оценить результат.

**Задача-минимум**: веб-страница, на которой пользователь вводит строку заголовка, а в ответ получает:
1) оценка заголовка,
2) найденные полезные индикаторы
3) подсказки, что далее делать с заголовком.

**Задача-максимум** (пока не решаем): генерация вариантов более качественных заголовков по тексту публикации или сочетанию чернового заголовка и краткого содержания.


# Инструментарий
- Python 3
- NLP-библиотека [spaCy 3.0](https://spacy.io/). Мы выбрали `spacy` так как это стабильная библиотека, ориентированная на конечное использование в коммерческих приложениях. Однако 3-я версия не очень хорошо зарекомендовала для себя для классификации большого набора данных, а transformers оказалась слишком долгой для обучения и медленной для задачи веб-сервиса. Поэтому мы использовали быстрый CatBoost.
- ML-библиотека CatBoost

Для разметки эталонного набора именованных сущностей использовалось [Label Studio](https://labelstud.io/). Вручную было размечено 3000 заголовков, далее эти результаты использовались для полуавтоматической разметки. Для разметки данных мы использовали стандартный формат [CoNLL format](https://www.signll.org/conll/) ([StackOverflow discussion](https://stackoverflow.com/questions/27416164/what-is-conll-data-format)).

In [50]:
from subprocess import PIPE, run
import json

# библиотеки обработки данных
import numpy as np
import pandas as pd
from scipy import stats

# прогрессбар
from tqdm.auto import tqdm
tqdm.pandas()

# natural language processing
import spacy

# ml models
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, minmax_scale, quantile_transform
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE

from catboost import Pool, CatBoostRegressor

# визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# фильтрация некритичных предупреждений pandas b jupyter
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore",
                      category=SettingWithCopyWarning)

DATASETS_PATH = '../DATASETS/title'

%matplotlib inline

# 1. Подготовка текстового корпуса

## 1.1. Изучим датасет, содержащий заголовки и число просмотров

С помощью веб-парсинга мы собрали большое количество данных в один общий датасет. 

In [ ]:
def read_df():
    df = pd.read_feather(f'{DATASETS_PATH}/total.feather')    
    df = df.set_index('url')  # feather doesn't work with str indices
    
    # parse timing cols ad datetime
    for col in ('post_time', 'parse_time'):
        df[col] = pd.to_datetime(df[col])
    
    return df

df = read_df()
df.head()

In [ ]:
df.info()

Посмотрим на распределение статей по источникам:

In [ ]:
df.source.value_counts()

## 1.2. ML-коррекция заниженного количества просмотров

Количество просмотров на сайтах иногда значительно отстает от ожидаемого или не всегда рассчитывается правильно.

Например, для новых статей или статей, изменивших статус публичности. Особенно это заметно, когда количество просмотров меньше количества лайков и закладок. Чтобы исправить такие значения, построим простую регрессионную модель на данных, которым мы можем доверять. Далее экстраполируем результат на «подозрительные» данные.

In [ ]:
df['timedelta'] = (df.parse_time - df.post_time).apply(lambda x: x.total_seconds())*1e-6
df_tmp = df[['likes_num', 'favs_num', 'comments_num', 'views_num', 'source', 'timedelta']].dropna()

# for categorical data (source feature)
df_tmp = pd.get_dummies(df_tmp)
df_tmp['suspicious'] = [False]*df_tmp.shape[0]
for col in ('likes', 'favs', 'comments'):
    df_tmp['suspicious'] += df_tmp[f'{col}_num'] > 0.1*df_tmp['views_num']

df_tmp_susp = df_tmp[df_tmp['suspicious'] == True]
df_tmp = df_tmp[df_tmp['suspicious'] == False]

df_tmp = df_tmp.drop(columns=['suspicious'])
df_tmp_susp = df_tmp_susp.drop(columns=['suspicious'])

y = df_tmp['views_num']
X = df_tmp.drop(columns=['views_num'])
reg = make_pipeline(StandardScaler(),
                    RandomForestRegressor(n_jobs=20))
reg.fit(X, y)
df_tmp_susp['views_num'] = reg.predict(df_tmp_susp.drop(columns=['views_num']))
df_tmp_susp['views_num'] = df_tmp_susp['views_num'].apply(round)
df_tmp = pd.concat([df_tmp, df_tmp_susp])

df.update(df_tmp)

Теперь мы можем исключить данные, которые не содержат числа просмотров:

In [ ]:
df = df.dropna(subset=['views_num'])

Далее мы не будем учитывать сайт, на котором размещена публикация, поэтому нормируем число просмотров для статей определенного источника на максимальное для каждого из них.

In [ ]:
for s in df.source.unique():
    cond = (df.source == s)
    m = df.loc[cond].views_num.max()
    df.loc[cond, 'views_num'] /= m

## 1.3. Отбор признаков для работы

**Число просмотров** — наша целевая переменная.

**Время от публикации до парсинга**. Важной характеристикой публикации является то, когда она была опубликована. Тематики статей меняются, растет число людей с доступом в интернет. Со временем статьи с актуальными темами продолжают получать дочитывания, а наиболее новые статьи еще не набрали своего. В результате мы имеем следующее распределение просмотров относительно шкалы времени:

In [ ]:
plt.scatter(-df.timedelta,
            df.views_num,
            s = 5,
            marker='.')
plt.xlabel('Время, усл. ед.')
plt.ylabel('Число просмотров.')

Для новых публикаций мы будем указывать нулевую отметку времени.

В результате мы используем сами тексты заголовков, число просмотров и интервал времени, за который число просмотров было набрано:

In [ ]:
Xy = df[['timedelta', 'title', 'views_num']].reset_index(drop=True)
Xy.views_num = Xy.views_num.fillna(0)

## 1.4. Предобработка числовых данных

С помощью z-оценки выкинем явные выбросы (оказалось, что это примерно 5 тыс. статей).

In [ ]:
Xy['z_score'] = stats.zscore(Xy['views_num'])
Xy = Xy.loc[Xy['z_score'].abs()<=3]
Xy = Xy.drop(columns=['z_score'])
Xy.reset_index(drop=True, inplace=True)
print(Xy.shape)

plt.scatter(-Xy.timedelta,
            Xy.views_num,
            s = 0.01,
            marker='.')
plt.xlabel('Время, усл. ед.')
plt.ylabel('Число просмотров.')
plt.show()

Гистограмма распределния числа просмотров: 

In [ ]:
plt.hist(Xy.views_num, bins=50)
plt.show()

Кривая имеет предсказуемый характер, поэтому мы можем воспользоваться  преобразованием [Quantile Transform](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.quantile_transform.html#sklearn.preprocessing.quantile_transform), чтобы трансформировать распределение к нормальному виду.

In [ ]:
y = Xy.views_num.to_numpy().reshape(-1, 1)
Xy['y'] = quantile_transform(y, output_distribution='normal')
plt.hist(Xy['y'], bins=200)
plt.show()

Для переменной времени применим обычную нормировку по интервалу, чтобы данные лежали в диапазоне `[0, 1]`:

In [ ]:
Xy.timedelta = minmax_scale(Xy.timedelta)
plt.hist(Xy.timedelta, bins=500)
plt.show()

# 2. Обучение модели распознавания именованных сущностей (модель NER)

В качестве базовой модели `spacy` мы используем `ru_core_news_lg`. В ней уже имеются сущности `PER` (персоналий), организаций (`ORG`), географических локаций (`LOC`). Кроме имеющихся именованных сущностей мы добавили в разметку следующие категории и подкатегории:

1. **Объект** `OBJ`. *О чём* эта статья.  Если заголовок состоит только из таких сущностей, значит перед нами что-то вроде статьи из словаря — объяснение сущности самого объекта. Примеры: "LESS: программируемый язык стилей", "Composer — менеджер зависимостей для PHP".
2. **Аудитория** `AUD`. Для кого написан этот текст. Примеры индикаторов аудитории: "для новичков, на Windows, профи, любой аккаунт, до 30 лет, русская версия" Аудитория выражается и просто через "я" — мы сравниваем себя с другими людьми через наш общий или различный опыт. Наиболее читаемые статьи обращаются к аудитории новичков, но это не значит, что их читают только новички. "Пайка для начинающих", "Hello World-проект на Flask", "Основы IP-телефонии", "Какой язык программирования стоит выучить пер
вым?".
3. **Польза**. Какую проблему показывает или решает публикация. В чём ее профит?
Польза может выражаться самыми разными способами:
  + **Маркеры типа текста** `TYPE`: инструкция ("как установить", "Шаблон базовой настройки маршрутизатора Cisco"), определение ("что такое... и с чем едят"), новость (Новое в Java 8"), личный опыт, сравнение объектов ("X или Y", "Python vs R") и т. д. По маркеру типа текста мы понимаем, с чем имеем дело.
  + **Указание числа используемых источников или рассматриваемых объектов** `NUM`: "10 лучших", "ТОП-3".
  + **Усилия и время, которые потратит читатель на саму статью или процесс** `EFFORT`: "За 15 минут, за один вечер, за один год, краткое руководство, в 11 строчек кода". Вполне возможно, что у человека достаточно времени, и он хочет детально во всём разобраться: "Подробно о..., всё про...". Главное, что вся нужная информация нашлась в одном месте.
  + **Маркеры последовательного подхода, нового типа изложения** `STRUCT`. В интернете не хватает структурированной информации, люди любят когда рассказывают "по порядку, детально, без воды".
  + **Предостережение об опасности или возможной ошибке** `DANGER`: "Проблема в ... и ее их решение", "Взлом... от которого не спасёт", " "X – ловушка для неопытных. Осторожно".
  + **Маркировка акта длинного повествования** `PART`. Указание части в заголовке подсказывает: перед нами часть большого текста. Хорошо работает следующий формат: "Общее название группы технологий. Часть N. Название технологии."  Примеры: "jQuery для начинающих. Часть 3. AJAX". "Bash-скрипты, часть 2: циклы". "Пишем игры на C++, Часть 1/3 — Написание мини-фреймворка", "Сети для самых маленьких. Часть шестая. Динамическая маршрутизация".
4. **Источник движения** — в хороших статьях заложена история путешествия, они приводят читателя из пункта А в пункт Б. Саму историю расскажет статья, но полезно прочертить вектор с помощью глагола, или если придется к месту — искренней эмоции.
  + **Побуждение к действию или само действие** `TODO`. Что мы будем делать в этой статье. "Пишем программу...", "настройка, обзор, запуск, ремонт". Примеры: "Извлекаем золото из старой электроники", "Запуск старых игр на Windows".
  + **Эмоция** `EMO`. С эмоциями не стоит перебарщивать, но иногда сильная эмоция или выражение отношения — то, что нужно. "Xудшее, что могло с нами случиться." "Почему научиться программировать так чертовски тяжело?". Помните: читатель не дурак, эмоции в заголовке работают только, если они неподдельные.

Однако нужно помнить, что каким бы ни был заголовок, главное – сам текст и внимательное отношение к читателю.

In [ ]:
def out(command):
    '''Выводит данные '''
    result = run(command, stdout=PIPE,
                 stderr=PIPE, universal_newlines=True,
                 shell=True)
    return result.stdout


try:
    nlp = spacy.load("./ner_model/model-best")
except OSError:
    # если модель ещё не обучалась
    # используем в качестве старта предобученную
    import ru_core_news_lg
    nlp = ru_core_news_lg.load()
    print(out("spacy init fill-config base_config.cfg config.cfg"))
    print(out("spacy convert ./train_data/title_ner.conll ./train_data/"))
    # это всего лишь прямой вызов из командной строки
    !spacy train config.cfg --output ./ner_model --paths.train ./train_data/title_ner.spacy --paths.dev ./train_data/title_ner.spacy

На нескольких примерах проверим корректность поиска именованных сущностей:

Теперь применим NER-модель к текстовой составляющей — признаку `title`. Определим, какие сущности (`ents`) встречаются и какова их доля (`fraction`) от общей длины заголовка:

In [2]:
def str_to_ents(s):
    seq = nlp(s).ents
    ents_list = [ent.label_ for ent in seq]
    try:
        fraction = len(''.join(str(ent) for ent in seq))/len(s)
    except ZeroDivisionError:
        fraction = 0.0
    return ents_list, fraction


try:
    # распознавание сущностей для большого массива
    # это трудоемкая операция, поэтому мы сохраняем результат
    # и загружаем его, если уже был проведен расчет
    Xy = pd.read_csv(f'{DATASETS_PATH}/Xy_ents.csv',
                     index_col=0)
    Xy.ents = Xy.ents.apply(eval)
except FileNotFoundError:
    Xy['ents'] = Xy.title.progress_apply(str_to_ents)
    Xy['ents_fraction'] = Xy['ents'].apply(lambda x: x[1])
    Xy['ents'] = Xy['ents'].apply(lambda x: x[0])
    Xy.to_csv(f'{DATASETS_PATH}/Xy_ents.csv')

Преобразуем полученные данные об именованных сущностях к числовому представлению.

In [3]:
# в файле tags.json хранятся данные 
with open(f'{DATASETS_PATH}/tags.json') as f:
    tags = json.load(f)

# определим, какие сущности мы зарезервировали
tag_names = [list(d.keys())[0] for d in tags]

def ents_to_array(ents_list):
    '''Преобразует список именованных сущностей
    для одного заголовка в массив чисел,
    где индекс соответствует номеру сущности,
    а число - количеству появлений в заголовке'''
    line = [0]*len(tag_names)
    for t in ents_list:
        try:
            i = tag_names.index(t)
            line[i] += 1
        except ValueError:
            pass
    return np.array(line, dtype=np.uint8)

ents_array = Xy.ents.progress_apply(ents_to_array)
data = np.array(ents_array.to_list())
tmp = pd.DataFrame(data, columns=tag_names)
Xy = pd.concat([Xy, tmp], axis = 1)
Xy.head()

,timedelta,title,views_num,y,ents,ents_fraction,PER,ORG,LOC,OBJ,...,SITE,ANNOUNCE,MATH,GAME,NEWS,CONF,COND,APP,LIB,FRAME
0,0.006861,Blackbox-сканеры в процессе оценки безопасност...,0.023319,1.959235,[DANGER],0.517241,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.006972,Инструменты управления командой разработки на ...,0.058947,2.606826,"[OBJ, OBJ, PUNCT, TYPE]",0.935897,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,0.006989,Стоит поиграть: обзор игры 7 Billion Humans,0.081565,2.938459,"[TYPE, PUNCT, TYPE, OS]",0.604651,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.007087,Как снизить расходы на разработку программного...,0.078663,2.899126,"[DANGER, DANGER, PUNCT]",0.932203,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.007103,Моя история в IT: от любви к математике до меж...,0.071457,2.783585,"[TYPE, PUNCT, TYPE, VERSION, TYPE]",0.829787,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Посмотрим на корреляцию признаков и числа просмотров `views_num` (без нелинейных преобразований `y`):

In [4]:
correlations = Xy.drop(columns=['y']).corr()
correlations['views_num'].sort_values(ascending=False)

views_num        1.000000
TYPE             0.067021
AUD              0.059252
NUM              0.058857
TECH             0.033479
ADV              0.028049
OS               0.027734
TODO             0.027263
ents_fraction    0.025298
APP              0.024423
EFFORT           0.022167
COND             0.020695
DEVICE           0.020341
EMO              0.015683
MATH             0.014419
OBJ              0.014397
STRUCT           0.014030
PART             0.013372
LANG             0.012273
LIB              0.009814
FRAME            0.009645
ORG              0.004831
GAME             0.001097
PER             -0.002634
NEWS            -0.004132
SITE            -0.004590
ANNOUNCE        -0.004949
CONF            -0.009781
DANGER          -0.010112
LOC             -0.013116
PROGRAM         -0.018553
timedelta       -0.144619
Name: views_num, dtype: float64

Относительно тегов мы можем предположить, что указание типа (`TYPE`),  используемой технологии (`TECH`), целевой аудитории `AUD` и совершаемого действия (`TODO`) обычно приводят к увеличению числа просмотров.

Указание опасности (`DANGER`), организации, персоны, локации (`ORG`, `PER`, `LOC`), конкретной программы (`PROGRAM`), конференции (`CONF`),видимо, в большей мере относятся к новостям, которые быстро теряют актуальности. Поэтому такие атрибуты связаны с уменьшением количества просмотров.

In [5]:
X = Xy.drop(columns=['title', 'views_num', 'ents', 'y'])
y = Xy.y

Сами корреляции коэффициенты корреляции в абсолютном значении невелики. Поэтому большее внимание надо отдавать языковой модели. Предварительно попробуем поискать композитные фичи, используя пары и тройки столбцов с тегами.

In [6]:
X_ents = X.drop(columns=['timedelta', 'ents_fraction'])
from itertools import combinations

cc2 = list(combinations(X_ents.columns,2))
X_ents2 = pd.concat([X_ents[c[0]].add(X_ents[c[1]]) for c in cc2],
          axis=1, keys=cc2)

cc3 = list(combinations(X_ents.columns,3))
X_ents3 = pd.concat([X_ents[c[0]].add(X_ents[c[1]]).add(X_ents[c[2]]) for c in cc3],
          axis=1, keys=cc3)

X_ents2.columns = X_ents2.columns.map('_'.join)
X_ents3.columns = X_ents3.columns.map('_'.join)
X_ents = pd.concat([X, X_ents2, X_ents3, Xy['views_num']], axis=1)

In [33]:
d = dict()
X_ents = X_ents.fillna(0)
X_ents['BOOL'] = X.ents_fraction.apply(lambda x: x == 0)
for col in X_ents.drop(columns=['views_num', 'timedelta', 'ents_fraction']).columns:
    d[col] = stats.pearsonr(X_ents['views_num'], X_ents[col])[0]

In [34]:
ner_list, value_list = [], []

for key in d:
    ner_list.append(set(key.split('_')))
    value_list.append(d[key])

In [35]:
df_ner_cor = pd.DataFrame.from_dict({'NER': ner_list, 'VALUE': value_list})
df_ner_cor.to_csv(f'{DATASETS_PATH}/df_ner_cor.csv')

In [49]:
df_ner_cor.sort_values(by='VALUE')

,NER,VALUE
4089,{BOOL},-0.029062
1429,"{PROGRAM, LOC, CONF}",-0.023704
1425,"{ANNOUNCE, PROGRAM, LOC}",-0.022861
1428,"{NEWS, PROGRAM, LOC}",-0.022713
1306,"{DANGER, PROGRAM, LOC}",-0.022632
...,...,...
1798,"{TYPE, TECH, AUD}",0.090052
1799,"{TYPE, AUD, OS}",0.090362
1796,"{TYPE, AUD, ADV}",0.090488
1794,"{TYPE, AUD, TODO}",0.091366


Метке `BOOL` соответствует полное отсутствие каких-либо сущностей. Таким образом, хотя бы минимальное соответствие, даже худшим по корреляциям из меток, лучше, чем ничего.

# 3. Обучение регрессионной модели

Мы привели текстовые данные к числовому представлению и теперь можем заняться задачей регрессии.

In [ ]:
X = X.fillna(0)
X_train, X_test, y_train, y_test = train_test_split(X_ents.drop(columns=['views_num']), y,
                                                    random_state=0)

При обучении используем структуру `Pool`:

In [ ]:
catboost_params = {
    'iterations': 10000,
    'learning_rate': 0.01,
    'eval_metric': 'RMSE',
    'early_stopping_rounds': 100,
    'use_best_model': True
}

def fit_model(X_train, y_train, X_test, y_test, catboost_params):
    train_pool = Pool(data = X_train,
                      label = y_train)

    validation_pool = Pool(data = X_test,
                           label = y_test)

    model = CatBoostRegressor(**catboost_params)

    return model.fit(train_pool,
                     eval_set=validation_pool,
                     verbose=100)


model = fit_model(X_train, y_train,
                  X_test, y_test,
                  catboost_params)

In [ ]:
print(MSE(model.predict(X_test), y_test))
plt.scatter(model.predict(X_test), y_test, s=0.01)
plt.show()

Вполне ожидаемо (и прискорбно), но регресионная модель обучается довольно слабо: на число просмотров оказывают влияния множество иных факторов, а не только сами сущности.

Использование трансформерной модели или других моделей, работающих с сутью текста, повысило бы качество, но усложнило бы коррекцию заголовка пользователем: балл оценки в таком случае сложно интерпретируется, возможны только действия "наугад". Однако такое решение можно использовать в будущем для генеративного подхода поиска заголовка.

Поэтому мы воспользуемся подходом программного определения правил, исходя из лучших сочетаний сущностей.

# 4. Упрощенная корреляционная модель

Напишем функцию, которая принимает на вход строку, а на выходе возвращает рекомендации для коррекции заголовка. Для этого нам понадобится несколько вспомогательных функций.

Проведем нормировку между 0 и 1.

In [56]:
df_ner_cor.VALUE = minmax_scale(df_ner_cor.VALUE)

In [57]:
df_ner_cor

,NER,VALUE
0,{PER},0.210073
1,{ORG},0.269408
2,{LOC},0.126752
3,{OBJ},0.345451
4,{AUD},0.701996
...,...,...
4085,"{COND, APP, LIB}",0.479847
4086,"{COND, APP, FRAME}",0.481229
4087,"{COND, FRAME, LIB}",0.408821
4088,"{APP, FRAME, LIB}",0.432201


In [ ]:
def ner_list_to_value(ner_list, df=df_ner_cor):
    return value